# Reference
+ https://www.cnblogs.com/Imageshop/p/3281703.html

# Question
## 什么是暗通道？
在绝大多数非天空的局部区域里，某些像素总会有某个颜色通道很低的值，用数学表达式定义的话就是$J^{dark}(x)=\min_{y\in \Omega (x)}(\min_{c\in \{r,g,b\}}J^c(y))$
，其中$J^c$表示彩色图像的每一个通道，$\Omega (x)$表示以像素x为中心的一个窗口。实际计算过程中可以理解为，首先对图像求出每一个像素RGB三通道中的最小值，然后进行以此最小值滤波,即可求出$J^{dark}$
## 暗通道有什么先验知识？
$J^{dark}-->0$
## 如何利用暗通道进行去雾？
首先了解雾图像形成模型：$I(x)=J(x)t(x)+A(1-t(x))$，其中$I(x)$为雾图,$J(x)$为无雾图，A是全球大气光成分，$t(x)$为透射率。

接下来就是推导过程：
<img src=./img/darkChannel.jpeg>
上述推论中都是假设全球达气光A值时已知的，在实际中，我们可以借助于暗通道图来从有雾图像中获取该值。具体步骤如下：

1、从**暗通道**图中按照亮度的大小取前0.1%的像素。

2、在这些位置中，在**原始有雾图像I**中寻找对应的具有最高亮度的点的值，作为A值。

到这一步，我们就可以进行无雾图像的恢复了。由上式可知：J=(I-A)/t+A  
现在I,A,t都已经求得了，因此，完全可以进行J的计算。

当投射图t的值很小时，会导致J的值偏大，从而使淂图像整体向白场过度，因此一般可设置一阈值T0，当t值小于T0时，令t=T0，本文中所有效果图均以T0=0.1为标准计算。
因此最终的恢复公式：
$J(x)=\frac{I(x)-A}{max(t(x),t_0)}+A$


In [5]:
import cv2;
import math;
import numpy as np;

# 求出暗通道
def DarkChannel(im,sz):
    """
    im:输入图像
    sz:窗口大小
    """
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

# 求出全球大气光值A
def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz);
    imvec = im.reshape(imsz,3);

    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx;
    return A

def TransmissionEstimate(im,A,sz):
    omega = 0.95;
    im3 = np.empty(im.shape,im.dtype);

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz);
    return transmission

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);

    return t;

def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]

    return res

if __name__ == '__main__':
    fn = 'T:\GitHub\DIP_lv\Experiment2\img\haze.png'


    src = cv2.imread(fn);

    I = src.astype('float64')/255;
 
    dark = DarkChannel(I,15);
    A = AtmLight(I,dark);
    te = TransmissionEstimate(I,A,15);
    t = TransmissionRefine(src,te);
    J = Recover(I,t,A,0.1);

    cv2.imshow("dark",dark);
    cv2.imshow("t",t);
    cv2.imshow('I',src);
    cv2.imshow('J',J);
    cv2.imwrite("./image/J.png",J*255);
    cv2.waitKey();

In [2]:
import cv2
import math
import numpy as np

#获取暗通道
def getDarkChannel(img,windowSize):
    """
    img:输入图像
    windowSize:窗口大小
    """
    b,g,r=cv2.split(img)
    darkC=cv2.min(cv2.min(r,g),b)
    """
    #cv2.getStructuringElement( ) 返回指定形状和尺寸的结构元素
    矩形:MORPH_RECT;
    交叉形:MORPH_CROSS;
    椭圆形:MORPH_ELLIPSE;
    第二和第三个参数分别是内核的尺寸以及锚点的位置
    """
    kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(windowSize,windowSize))
    """
    二值图像f(x,y) 的膨胀操作，类似于对图像的卷积操作.
    需要有个 kernel 操作矩阵，类似于卷积核(filters, kernel)，常见的是 3X3 的矩阵. 这是形态学处理的核心.
    但与卷积不同的是，如果矩阵中的像素点有任意一个点的值是前景色，则设置中心像素点为前景色；否则不变.
    最小值卷积相当于腐蚀操作
    """
    darkChannel=cv2.erode(darkC,kernel)
    return darkChannel

# 求出全球大气光值
def AtmLight(img,darkChannel):
    """
    img:图像
    darkChannel: 暗通道
    """
    [h,w]=img.shape[:2]
    imgSize=h*w
    numpx = int(max(math.floor(imgSize/1000),1))
    darkvec = dark.reshape(imgSize)
    imvec = img.reshape(imgSize,3)

    indices = darkvec.argsort()
    indices = indices[imgSize-numpx::]

    atmsum = np.zeros([1,3])
    for channel_index in range(1,numpx):
       atmsum = atmsum + imvec[indices[channel_index]]

    A = atmsum / numpx
    return A
# 估计t值
def TransmissionEstimate(im,A,sz):
    omega = 0.95
    im3 = np.empty(im.shape,im.dtype)
    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*getDarkChannel(im3,sz)
    return transmission

# 导向滤波
def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r))
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r))
    cov_Ip = mean_Ip - mean_I*mean_p

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r))
    var_I   = mean_II - mean_I*mean_I

    a = cov_Ip/(var_I + eps)
    b = mean_p - a*mean_I

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r))
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r))

    q = mean_a*im + mean_b
    return q;
# 进一步修正t值。soft matting方法，能够得到非常细腻的结果。采用导向滤波的方法。
def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray)/255
    r = 60
    eps = 0.0001
    t = Guidedfilter(gray,et,r,eps)

    return t

def recover(img,t,A,tx=0.1):
    """
    img:雾图像
    t: 投射图
    A: 大气光值
    tx: 投射图阈值
    """
    result=np.empty(img.shape,img.dtype)
    t=cv2.max(t,tx)

    for channel_index in range(0,3):
        result[:,:,channel_index]=(img[:,:,channel_index]-A[0,channel_index])/t+A[0,channel_index]
    
    return result

if __name__ == '__main__':
    fn = r'.\img\haze.png'


    src = cv2.imread(fn);

    I = src.astype('float64')/255
 
    dark = getDarkChannel(I,15)
    A = AtmLight(I,dark)
    te = TransmissionEstimate(I,A,15)
    t = TransmissionRefine(src,te)
    clear_output = recover(I,t,A,0.1)

    cv2.imshow("dark",dark)
    cv2.imshow("t",t)
    cv2.imshow('I',src)
    cv2.imshow('J',clear_output)
    cv2.imwrite(r".\img\J_te.png",clear_output*255)
    cv2.waitKey()